In [ ]:
%serialconnect to port=/dev/tty.usbserial-0001 --baud=115200

Found serial ports: /dev/cu.usbserial-0001, /dev/cu.Bluetooth-Incoming-Port, /dev/cu.wlan-debug 
Connecting to --port=/dev/cu.usbserial-0001 --baud=115200 


<thonny>{'sys': ("<module 'sys'>", 1061266632), 'os': ("<module 'uos'>", 1061217372), '_': ("['__init__.py', 'sensor.py']", 1073647616), 'gc': ("<module 'gc'>", 1061265036), 'bdev': ('<Partition type=1, subtype=129, address=2097152, s...', 1073630896), 'uos': ("<module 'uos'>", 1061217372)}</thonny>>

[too many x04s 5]Ready.


In [32]:
from machine import Pin, SoftI2C
from time import sleep_ms
import os


mpu_addr = 0x68
mpu_pwr_mgmt = 0x6b

mpu_accel_x_h = 0x3b
mpu_accel_x_l = 0x3c
mpu_accel_y_h = 0x3d
mpu_accel_y_l = 0x3e
mpu_accel_z_h = 0x3f
mpu_accel_z_l = 0x40

mpu_gyro_x_h = 0x43
mpu_gyro_x_l = 0x44
mpu_gyro_y_h = 0x45
mpu_gyro_y_l = 0x46
mpu_gyro_z_h = 0x47
mpu_gyro_z_l = 0x48

mpu_gyro_sel = 340.0
mpu_accel_sel = 16348.0

class mpu6050():
    def __init__(self, i2c):
        self.i2c = i2c
        
    def power(self):
        self.i2c.writeto_mem(mpu_addr, mpu_pwr_mgmt, bytes([0]))

    def combine_register_values(self, h, l):
        if not h[0] & 0x80:
            return h[0] << 8 | l[0]
        return -((h[0] ^ 255) << 8) |  (l[0] ^ 255) + 1

    def get_accel(self):
        accel_x_h = self.i2c.readfrom_mem(mpu_addr, mpu_accel_x_h, 1)
        accel_x_l = self.i2c.readfrom_mem(mpu_addr, mpu_accel_x_l, 1)
        accel_y_h = self.i2c.readfrom_mem(mpu_addr, mpu_accel_y_h, 1)
        accel_y_l = self.i2c.readfrom_mem(mpu_addr, mpu_accel_y_l, 1)
        accel_z_h = self.i2c.readfrom_mem(mpu_addr, mpu_accel_z_h, 1)
        accel_z_l = self.i2c.readfrom_mem(mpu_addr, mpu_accel_z_l, 1)

        return [self.accel_sel(self.combine_register_values(accel_x_h, accel_x_l)),
                self.accel_sel(self.combine_register_values(accel_y_h, accel_y_l)),
                self.accel_sel(self.combine_register_values(accel_z_h, accel_z_l))]

    def accel_sel(self, accel):
        return accel / mpu_accel_sel

    def get_gyro(self):
        gyro_x_h = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_x_h, 1)
        gyro_x_l = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_x_l, 1)
        gyro_y_h = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_y_h, 1)
        gyro_y_l = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_y_l, 1)
        gyro_z_h = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_z_h, 1)
        gyro_z_l = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_z_l, 1)

        return [self.gyro_sel(self.combine_register_values(gyro_x_h, gyro_x_l)),
                self.gyro_sel(self.combine_register_values(gyro_y_h, gyro_y_l)),
                self.gyro_sel(self.combine_register_values(gyro_z_h, gyro_z_l))]

    def gyro_sel(self, gyro):
        return gyro / mpu_gyro_sel


if __name__ == "__main__":
    mpu = mpu6050(SoftI2C(scl=Pin(23), sda=Pin(22)))
    
    while True:
        print("Accelerometer:\t", mpu.get_accel(), "g")
        print("Gyroscope:\t", mpu.get_gyro(), "°/s")
        sleep_ms(1000)


Accelerometer:	 [0.4161977, 0.1932958, 1.007585] g
Gyroscope:	 [3.047059, 190.9853, 191.6118] °/s
Accelerometer:	 [0.4093467, 0.1923171, 1.014681] g
Gyroscope:	 [3.111765, 190.9059, 191.7618] °/s
Accelerometer:	 [0.4137509, 0.189381, 1.00367] g
Gyroscope:	 [3.102941, 190.9353, 191.6823] °/s
Accelerometer:	 [0.4103254, 0.1920724, 1.010276] g
Gyroscope:	 [3.173529, 190.9353, 191.6382] °/s


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 74, in <module>
KeyboardInterrupt: 


In [33]:
from machine import Pin, SoftI2C
from time import sleep_ms

import os

mpu_addr = 0x68
mpu_pwr_mgmt = 0x6B

mpu_accel_x_h = 0x3B
mpu_accel_x_l = 0x3C
mpu_accel_y_h = 0x3D
mpu_accel_y_l = 0x3E
mpu_accel_z_h = 0x3F
mpu_accel_z_l = 0x40

mpu_gyro_x_h = 0x43
mpu_gyro_x_l = 0x44
mpu_gyro_y_h = 0x45
mpu_gyro_y_l = 0x46
mpu_gyro_z_h = 0x47
mpu_gyro_z_l = 0x48

mpu_lsbc = 340.0
mpu_lsbg = 16348.0

g = 9.8066

def mpu_init(i2c):
    i2c.writeto_mem(mpu_addr, mpu_pwr_mgmt, bytes([0]))

def combine_register_values(h, l):
    if not h[0] & 0x80:
        return h[0] << 8 | l[0]
    return -((h[0] ^ 255) << 8) |  (l[0] ^ 255) + 1

def mpu_get_accel(i2c):
    accel_x_h = i2c.readfrom_mem(mpu_addr, mpu_accel_x_h, 1)
    accel_x_l = i2c.readfrom_mem(mpu_addr, mpu_accel_x_l, 1)
    accel_y_h = i2c.readfrom_mem(mpu_addr, mpu_accel_y_h, 1)
    accel_y_l = i2c.readfrom_mem(mpu_addr, mpu_accel_y_l, 1)
    accel_z_h = i2c.readfrom_mem(mpu_addr, mpu_accel_z_h, 1)
    accel_z_l = i2c.readfrom_mem(mpu_addr, mpu_accel_z_l, 1)

    return [accel_correction(combine_register_values(accel_x_h, accel_x_l)),
            accel_correction(combine_register_values(accel_y_h, accel_y_l)),
            accel_correction(combine_register_values(accel_z_h, accel_z_l))]

def accel_correction(accel):
    return accel / mpu_lsbg

def mpu_get_gyro(i2c):
    gyro_x_h = i2c.readfrom_mem(mpu_addr, mpu_gyro_x_h, 1)
    gyro_x_l = i2c.readfrom_mem(mpu_addr, mpu_gyro_x_l, 1)
    gyro_y_h = i2c.readfrom_mem(mpu_addr, mpu_gyro_y_h, 1)
    gyro_y_l = i2c.readfrom_mem(mpu_addr, mpu_gyro_y_l, 1)
    gyro_z_h = i2c.readfrom_mem(mpu_addr, mpu_gyro_z_h, 1)
    gyro_z_l = i2c.readfrom_mem(mpu_addr, mpu_gyro_z_l, 1)

    return [combine_register_values(gyro_x_h, gyro_x_l) / mpu_lsbc,
            combine_register_values(gyro_y_h, gyro_y_l) / mpu_lsbc,
            combine_register_values(gyro_z_h, gyro_z_l) / mpu_lsbc]

if __name__ == "__main__":
    i2c = SoftI2C(scl=Pin(23), sda=Pin(22))
    mpu_init(i2c)
    
    while True:
        print("Accelerometer:\t", mpu_get_accel(i2c), "g")
        print("Gyroscope:\t", mpu_get_gyro(i2c), "°/s")
        sleep_ms(1000)

Accelerometer:	 [0.415953, 0.1896256, 1.00734] g
Gyroscope:	 [2.914706, -0.9970588, -0.2058824] °/s
Accelerometer:	 [0.4161977, 0.1886469, 1.013213] g
Gyroscope:	 [3.111765, -1.223529, -0.4735294] °/s
Accelerometer:	 [0.4137509, 0.1891363, 1.017128] g
Gyroscope:	 [2.991176, -1.305882, -0.3911765] °/s
Accelerometer:	 [0.4135062, 0.1849768, 1.017861] g
Gyroscope:	 [3.167647, -1.111765, -0.2323529] °/s
Accelerometer:	 [0.415219, 0.1925618, 1.011989] g
Gyroscope:	 [3.117647, -1.105882, -0.5764706] °/s
Accelerometer:	 [0.4157084, 0.1879129, 1.006117] g
Gyroscope:	 [3.117647, -1.194118, -0.3764706] °/s
Accelerometer:	 [0.4068999, 0.1822853, 1.009542] g
Gyroscope:	 [3.108824, -1.255882, -0.35] °/s
Accelerometer:	 [0.409102, 0.1891363, 1.012234] g
Gyroscope:	 [3.120588, -1.223529, -0.4588235] °/s
Accelerometer:	 [0.4127722, 0.1864448, 1.013702] g
Gyroscope:	 [3.047059, -1.244118, -0.4147058] °/s
Accelerometer:	 [0.4188891, 0.1906044, 1.009298] g
Gyroscope:	 [3.297059, -1.164706, -0.3882353] °/

Traceback (most recent call last):
  File "<stdin>", line 71, in <module>
KeyboardInterrupt: 
